In [58]:
import torch
import numpy as np 

In [59]:
with open('wizard_of_oz.txt','r',encoding='utf-8')as f :
    text = f.read()
# print (text[:200])

In [60]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
block_size = 8

# encoder and decoder 


In [61]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i ,ch in enumerate(chars)}

encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

data = torch.tensor(encode(text), dtype = torch.long)



In [62]:
data = torch.tensor(encode(text), dtype = torch.long)

In [63]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]


In [64]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([80]) the target: 1
when input is tensor([80,  1]) the target: 1
when input is tensor([80,  1,  1]) the target: 28
when input is tensor([80,  1,  1, 28]) the target: 39
when input is tensor([80,  1,  1, 28, 39]) the target: 42
when input is tensor([80,  1,  1, 28, 39, 42]) the target: 39
when input is tensor([80,  1,  1, 28, 39, 42, 39]) the target: 44
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) the target: 32


In [65]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [66]:
x_b, y_b = get_batch('train')


In [67]:
from torch.nn.modules.loss import F


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self,index,targets=None):
        logits = self.token_embedding_table(index)
        if targets == None:
            loss = None 
        else:
            Batch , Time ,Channels = logits.shape
            logits = logits.view(Batch*Time,Channels)
            targets = targets.view(Batch*Time)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss 
    def generate (self , idx ,max_new_tokens):
        for i in range (max_new_tokens):
            logits, target = self(idx)
            logits = logits[:,-1,:]
            probs = F.softmax(logits,dim =-1)
            next_idx = torch.multinomial(probs,num_samples=1)
            idx = torch.cat((idx,next_idx),dim=1)
        return idx
    



In [68]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(x_b,y_b)
print(logits.shape)
print(loss)

print(decode(m.generate(idx= torch.zeros((1,1),dtype = torch.long), max_new_tokens = 100)[0].tolist()))


torch.Size([32, 81])
tensor(4.9362, grad_fn=<NllLossBackward0>)

:iaNhM?ag1uGaz("wrn)!y﻿pg8DlqV349z﻿G﻿40A*h8
LNJZ&8?)3Q,X
n4nYv2(XF4)DN33OEgMilndMLpO7I33Q[(j:g13 vwS
